In [1]:
import torch
import numpy as np
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn as nn
from torch.autograd import Variable

In [16]:
1 + torch.randn(*x.size())


(0 ,.,.) = 
  2.5016  1.2135  1.1183
  0.2357  1.1116  1.7245

(1 ,.,.) = 
 -0.1776  0.3805  2.1849
  1.8267  1.5925  1.7707
[torch.FloatTensor of size 2x2x3]

In [17]:
x = torch.Tensor([[1, 2], [3, 4]])
x


 1  2
 3  4
[torch.FloatTensor of size 2x2]

In [19]:
a = torch.Tensor([1, 2])
x * a


 1  4
 3  8
[torch.FloatTensor of size 2x2]

In [11]:
x = torch.Tensor([[[1, 2, 3], [4, 5, 6]], [[1, 2, 3], [4, 5, 6]]])
x.size()

torch.Size([2, 2, 3])

In [17]:
np.prod(x.size()[1:])

6

In [4]:
tr = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])

In [14]:
batch_size = 1

train_dataset = dsets.MNIST(root='../data',
                            train=True,
                            transform=tr,
                            download=True)

test_dataset = dsets.MNIST(root='../data',
                           train=False,
                           transform=transforms.ToTensor())

In [17]:
type(train_dataset.train_labels)

torch.LongTensor

In [ ]:
t.__dict__

In [12]:
# Data Loader (Input Pipeline)
x_train_shape = train_dataset.train_data.shape
x_test_shape = test_dataset.test_data.shape
# train_dataset.train_data = train_dataset.train_data / 256.
# test_dataset.test_data = test_dataset.test_data.float() / 256.

mean_img = train_dataset.train_data.mean(dim=0)
# train_dataset.train_data -= mean_img
# test_dataset.test_data -= mean_img

# random sampler
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

AttributeError: 'torch.ByteTensor' object has no attribute 'mean'

In [9]:
for x, y in train_loader:
    break

In [11]:
x.size()

torch.Size([1, 784])

In [4]:
train_dataset.train_data = train_dataset.train_data[:2]

In [5]:
type(train_dataset.train_data[0])

torch.FloatTensor

In [6]:
train_dataset.train_data[0:2].shape

torch.Size([2, 28, 28])

In [7]:
for (inp, lab) in train_loader:
    print(inp.shape)
    print(lab.shape)

torch.Size([1, 1, 28, 28])
torch.Size([1])
torch.Size([1, 1, 28, 28])
torch.Size([1])


In [60]:
class WeightNorm(nn.Module):
    def __init__(self, in_features, out_features, alpha):
        super(WeightNorm, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        
        self.internal_weight = nn.Parameter(
            torch.FloatTensor(in_features, out_features).cuda(), requires_grad=True)
        self.internal_weight.data.normal_(0, 0.01)
        self.magnitude = nn.Parameter(
            torch.FloatTensor(out_features).cuda(), requires_grad=True)
        self.magnitude.data.fill_(1)
        self.bias = nn.Parameter(
            torch.FloatTensor(out_features).cuda(), requires_grad=True)
        self.bias.data.fill_(0)
        self.sampled = Variable(torch.FloatTensor(self.internal_weight.data.shape).cuda(), requires_grad=False)
        self.sampled.data.fill_(0)

        self.parameters = [self.internal_weight, self.magnitude, self.bias]

    def forward(self, input, deterministic=False, tangent=False):
        W_norm = self.internal_weight / self.internal_weight.norm(dim=0)
        if not deterministic:
            self.sampled.data.normal_(0, np.sqrt(self.alpha))
        else:
            self.sampled.data.fill_(0)
        if tangent:
            coef = W_norm.mul(self.sampled).sum(dim=1)
            projected = self.sampled - W_norm * coef.view(-1, 1)
        else:
            projected = self.sampled
        W_noisy = W_norm + projected
        W_noisy_normed = W_noisy / W_noisy.norm(dim=0)
        self.weight = (self.magnitude * W_noisy_normed)
        result = input.mm(self.weight)
        return result.add(self.bias)
    
    def __repr__(self):
        params = ''
        if self._parameters:
            params = ', '.join(list(self._parameters.keys()))
            params = '\t' + params
        return self.__class__.__name__ + ' (' \
            + str(self.in_features) + ' -> ' + str(self.out_features) + ')' + params

class Linear(nn.Linear):
    def __repr__(self):
        params = ''
        if self._parameters:
            params = ', '.join(list(self._parameters.keys()))
            params = '\t' + params
        return self.__class__.__name__ + ' (' \
            + str(self.in_features) + ' -> ' + str(self.out_features) + ')' + params
        

class FC_Net(nn.Module):
    def __init__(self, input_size, num_classes, nonlinearity=nn.ReLU()):
        super(FC_Net, self).__init__()
        self.nonlinearity = nonlinearity

        self.fc1 = Linear(input_size, 1024)
        self.fc1.weight.data.normal_(mean=0, std=0.01)
        
        self.fc2 = Linear(1024, 1024)
        self.fc2.weight.data.normal_(mean=0, std=0.01)
        
        self.fc3 = Linear(1024, 2048)
        self.fc3.weight.data.normal_(mean=0, std=0.01)
        
        self.fc4 = Linear(2048, num_classes) 
        self.fc4.weight.data.normal_(mean=0, std=0.01)
    
    def forward(self, x):
        out = x.view(x.size(0), np.prod(x.size()[1:]))
        out = self.fc1(x)
        out = self.nonlinearity(out)
        out = self.fc2(x)
        out = self.nonlinearity(out)
        out = self.fc3(x)
        out = self.nonlinearity(out)
        out = self.fc4(x)
        return out


class WN_Net(nn.Module):
    def __init__(self, input_size, num_classes, nonlinearity=nn.ReLU(), alpha=0.01):
        super(WN_Net, self).__init__()
        self.nonlinearity = nonlinearity
        self.fc1 = WeightNorm(input_size, 1024, alpha)
        self.fc2 = WeightNorm(1024, 1024, alpha)
        self.fc3 = WeightNorm(1024, 2048, alpha)
        self.fc4 = WeightNorm(2048, num_classes, alpha)

    def forward(self, x, deterministic=False):
        hid1 = self.fc1.forward(x, deterministic=deterministic)
        out1 = self.nonlinearity(hid1)
        hid2 = self.fc2.forward(out1, deterministic=deterministic)
        out2 = self.nonlinearity(hid2)
        hid3 = self.fc3.forward(out2, deterministic=deterministic)
        out3 = self.nonlinearity(hid3)
        hid4 = self.fc4.forward(out3, deterministic=deterministic)
        return hid4

In [58]:
a = WN_Net(28*28, 10)
for p in a.parameters():
    print(type(p))

<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>


In [61]:
a = FC_Net(28*28, 10)
for p in a.parameters():
    print(type(p))

<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
